In [1]:
from ulkb import *   # import ULKB Logic namespace
settings.serializer.ulkb.ensure_ascii = False

a = TypeVariable('a') # a : *

%set_env ACE_GRAMMAR erg.dat

import mrs_logic

ctx = mrs_logic.get_current_context()
ctx.options['to_ulkb']['universe_type'] = a   # set the type of the (FOL) universe to "a"
ctx.options['to_ulkb']['drop_uvars'] = True   # delete "u" vars
ctx.options['to_ulkb']['drop_ivars'] = True   # delete "i" vars
ctx.options.to_ulkb.mk_q_exists = ['_the_q']
ctx.options.to_ulkb.mk_q_forall = ['udef_q']

from delphin import itsdb, mrs
from delphin.codecs import simplemrs, mrx, mrsprolog
import tree_algorithm

env: ACE_GRAMMAR=erg.dat


In [2]:
sentences = ['let us go to the store.', 'I am going to store, lets go.']
sentences = ['Every child sings.', 'All the children sing.','All children sing.',
             'Some children sing.','Some children sing.', 'Children sing.','The child sings.']
sentences = ['My friend and neighbor often helps me.',
             'it is not the case that every representative of a company saw a sample.']
sentences = ['every dog that barks is happy','every dog that is happy barks']
sentences = ['John and Mary are carrying a piano']
sentences = ['Charles hates no one that Agatha hates.'] 
txt = "The cat is white."

sol = mrs_logic.solve1(txt)
res = sol.to_ulkb()
res

('h1', {'h4': [<EP object (h4:_the_q(ARG0 x3, RSTR h5, BODY h6)) at 4445427584>], 'h7': [<EP object (h7:_cat_n_1(ARG0 x3)) at 4445426432>], 'h1': [<EP object (h1:_white_a_1(ARG0 e2, ARG1 x3)) at 4445422400>]})


(∃ x3, _cat_n_1 x3 ∧ (∃ e2, _white_a_1 e2 x3)) : 𝔹

In [ ]:
# https://pypi.org/project/fastcoref/

from fastcoref import FCoref

model = FCoref()

txt = ['Someone who lives in Dreadbury killed Agatha.',
       'Agatha, the butler, and Charles live in Dreadbury, and are the only people who live therein.',
       'A killer always hates his victim, and is never richer than his victim',
       'Charles hates no one that Aunt Agatha hates.',
       'Agatha hates everyone except the butler',
       'The butler hates everyone not richer than Aunt Agatha',
       'The butler hates everyone Aunt Agatha hates',
       'No one hates everyone',
       'Agatha is not the butler']

preds = model.predict(texts= txt)
preds

In [ ]:
from fastcoref import spacy_component
import spacy

text = 'The nurse notified the patient that his shift would be ending in an hour.'

nlp = spacy.load("en_core_web_sm")
nlp.add_pipe("fastcoref")

doc = nlp(text)
for cluster in doc._.coref_clusters:
    for a,b in cluster:
        print(text[a:b])

In [ ]:
res = []
for m in open("ws201.txt"):
    mrs = simplemrs.decode(m)
    try:
        s = mrs_logic.Solver(mrs)
        res.append(s.count_solutions())
    except:
        continue
print(res)

In [ ]:
from delphin import itsdb, mrs
from delphin.codecs import simplemrs, mrx, mrsprolog
import tree_algorithm

sentences = ['Every child sings.', 'All the children sing.','All children sing.',
             'Some children sing.','Some children sing.',
             'Children sing.','The child sings.']
sentences = ['My friend and neighbor often helps me.']
sentences = ['dogs that bark are happy','dogs that are happy barks']

res, ers = [], []
for txt in sentences:
    for i,mrs in enumerate(mrs_logic.parse(txt)):
        solver = mrs_logic.Solver(mrs)
        with open(f"{i}.mrs.pl", "w") as fp:
            print(mrsprolog.encode(mrs, indent=True), file = fp)
        n = solver.count_solutions()
        print(f"reading:{i} solutions:{n}")
        for sol in solver.iterate_solutions():
            res = sol.to_ulkb()
            print(res.serialize(show_types=False))